<header>
    <h1>CA4010 - Data warehousing and Data mining</h1>
    <h2>Continuous assessment project</h2>
</header>
<p>
    For this project, we want to predict if a project submitted to 
    <a href="https://www.indiegogo.com">indiegogo.com</a> will or will not be funded.
    For this purpose, we'll use a
    <a href="https://www.kaggle.com/kingburrito666/indiegogo-project-statistics/data">
    dataset from kaggle containing one year of indiegogo projects.</a>
    The version used here is the concatenation of all csv files using the given 'combiner.py'.<br/>
    However, some modifications has been made from combiner.py:
    <ul>
        <li>
            One row was normalized : <i>in_forever_funding</i> was passed to 'True' instead of 'null' to avoid
            type warning at loading time and use boolean type for this attribute.
        </li>
        <li>
            The file was saved as tsv (tab separated values) instead of csv for a better readability.
        </li>
    </ul>
</p>
<p>
    This notebook will decribe and show how we'll clean this dataset and what will be the attributes
    of the dataset we will use for all our further analysis.
</p>

<h2>Overview of the dataset</h2>
<p>
    We'll first detail the dataset as it is and analyze it.
</p>

In [30]:
import pandas as pd

In [31]:
infile = 'indiegogo_raw_data.tsv'
dataset = pd.read_csv(infile, sep='\t')
dataset.shape

(1720022, 21)

<p>
    We can see that the dataset contains <b>21 attributes</b>, which is way too much.
    We'll have to remove the less interesting ones and keep between <b>6 and 8 attributes</b>.
    It contains also more that <b>1 millions projects</b> which seems to be a very high number of 
    projects for only one year.
</p>

In [3]:
dataset[:5]

,id,title,nearest_five_percent,tagline,cached_collected_pledges_count,igg_image_url,compressed_image_url,balance,currency_code,amt_time_left,...,category_url,category_name,category_slug,card_type,collected_percentage,partner_name,in_forever_funding,friend_contributors,friend_team_members,source_url
0,773971,"Lasers, Lasers everywhere! Then I can make som...",100,Down The Rabbit Hole is wanting to expand into...,43,https://c1.iggcdn.com/indiegogo-media-prod-cld...,https://c1.iggcdn.com/indiegogo-media-prod-cld...,?520,GBP,No time left,...,/explore/small_business,Small Business,small_business,project,104%,null,False,[],[],https://www.indiegogo.com/explore#/browse/landing
1,1820996,Support Kanekta!,100,"Support the creation of Kanekta, every $1 you ...",11,https://c1.iggcdn.com/indiegogo-media-prod-cld...,https://c1.iggcdn.com/indiegogo-media-prod-cld...,$511,CAD,No time left,...,/explore/small_business,Small Business,small_business,project,102%,null,False,[],[],https://www.indiegogo.com/explore#/browse/landing
2,335364,Maise Designs at PCC!,100,"I've found myself, due to family sicknesses, n...",6,https://c1.iggcdn.com/indiegogo-media-prod-cld...,https://c1.iggcdn.com/indiegogo-media-prod-cld...,$510,USD,No time left,...,/explore/small_business,Small Business,small_business,project,102%,null,False,[],[],https://www.indiegogo.com/explore#/browse/landing
3,84385,Moved by Design,100,Moved by Design offers a holistic service that...,100,https://c1.iggcdn.com/indiegogo-media-prod-cld...,https://c1.iggcdn.com/indiegogo-media-prod-cld...,$507,USD,No time left,...,/explore/small_business,Small Business,small_business,project,101%,null,False,[],[],https://www.indiegogo.com/explore#/browse/landing
4,613091,Help Local Artisans Become Official Non-Profit...,100,Help our group of artisans obtain legal status...,15,https://c1.iggcdn.com/indiegogo-media-prod-cld...,https://c1.iggcdn.com/indiegogo-media-prod-cld...,$505,USD,No time left,...,/explore/small_business,Small Business,small_business,project,101%,null,False,[],[],https://www.indiegogo.com/explore#/browse/landing


<h2>Removing uneeded projects</h2>
<p>
As we want to know if a project will be funded or not, some projects contained in this dataset are not relevant for us. We will remove all projects which are:
    <ul>
        <li>Funded for ever</li>
        <li>Still running</li>
        <li>Duplicated</li>
    </ul>
</p>
<p>We first remove <b>forever funding projects</b> which are not relevant for our needs</p>

In [32]:
dataset = dataset[dataset.in_forever_funding != True]
dataset.shape

(1553639, 21)

<p>We're not interested by funding percentage of projects which are <b>still running</b></p>

In [33]:
dataset = dataset[dataset.amt_time_left == 'No time left']
dataset.shape

(1407807, 21)

<p>Now that we have removed irrelevant projects, let's check if this dataset contains <b>duplicated projects</b></p>

In [34]:
dataset.shape[0] - len(set(dataset['id']))

1261489

<p>
    This dataset contains more than <b>1 million of duplicated projects</b>,
    that is to say the <b>majority of this dataset</b> is project duplication !<br/>
    Let's remove duplicated projects.
</p>

In [35]:
dataset = dataset.drop_duplicates(subset=['id'], keep='last')
dataset.shape

(146318, 21)

<h2>Cleaning values</h2>
<p>
    Some values have a numerical and a string part.
    As they are not usable as it is, we want transform them into <b>numerical values</b> and get rid of the string part.
    This transformation concern:
    <ul>
        <li>balance (the amount given by the crowdfunders)</li>
        <li>collected percentage</li>
    </ul>
</p>

In [36]:
dataset.balance = dataset.balance.apply(lambda x: ''.join(c for c in x if c.isdigit()))
dataset.balance = dataset.balance.apply(pd.to_numeric)

<p>Now, we'll check if the other balance values have all the same currency.</p>

In [37]:
set(dataset.currency_code)

{'AUD', 'CAD', 'EUR', 'GBP', 'USD'}

<p>
    We can see that the currency of balance can vary.
    For consistency, we will convert all currencies to <b>USD</b>.
</p>

In [38]:
change_currencies = {'AUD': 0.78, 'CAD': 0.80, 'EUR': 1.18, 'GBP': 1.33, 'USD': 1}
dataset.balance = dataset.apply(lambda row: change_currencies[row['currency_code']] * row['balance'], axis=1)

<p>Then we convert percentage values removing the '%' sign.</p>

In [39]:
dataset.collected_percentage = dataset.collected_percentage.apply(lambda x: ''.join(c for c in x if c.isdigit()))
dataset.collected_percentage = dataset.collected_percentage.apply(lambda x: pd.to_numeric(x, downcast='float'))

<p>As we have the amount of fund collected and the collected percentage, it will be interesting to compute the <b>targeted amount</b> of each project. As we can't know the targeted amount of projects with <b>collected_percentage of 0</b>, theses projects will unfortunately won't be usable. Indeed, the amount required is probably one of the most important keys of project success. So we'll remove the projects with collected percentage of 0</p>

In [40]:
dataset = dataset[dataset.collected_percentage > 0]

In [49]:
dataset['target_amount'] = dataset.apply(lambda row: (row['balance'] * 100) / row['collected_percentage'], axis=1)

<p>
    Some other values are not usable as they are because they are to complex, such as:
    <ul>
        <li>title</li>
        <li>tagline</li>
        <li>partner_name</li>
    </ul>
</p>
<p>
    <b>Title</b> and <b>tagline</b> are still very interesting values which may 
    influence the success of a campaign.
    We can use the <b>number of character</b> of each one to see if a short or long title (or tagline)
    has an impact on the success of a campaign.
</p>

In [53]:
dataset.title = dataset.title.apply(lambda x: len(str(x)))

In [54]:
dataset.tagline = dataset.tagline.apply(lambda x: len(str(x)))

<p>
    <b>Partner name</b> is not usable as it is because searching for similar partner_name won't
    be of a great help for our analysis. However knowing if having a partner help a campaign to succeed is
    much more usable. Let's convert the partner_name values to <b>boolean values</b>:
    <ul>
        <li><b>True</b> if the campaign owner has one or more partners</li>
        <li><b>False</b> otherwise</li>
    </ul>
</p>

In [55]:
dataset.partner_name = dataset.partner_name.apply(lambda x: False if x == 'null' else True)

<p>
    Same problem with <b>friend_team_members</b> and <b>friend_contributors</b>
    Let's convert these to <b>boolean values</b>:
    <ul>
        <li><b>True</b> if the campaign owner has one or more friend which contribute or in its team</li>
        <li><b>False</b> otherwise</li>
    </ul>
</p>

In [56]:
dataset.friend_team_members = dataset.friend_team_members.apply(lambda x: False if x == '[]' else True)
dataset.friend_contributors = dataset.friend_contributors.apply(lambda x: False if x == '[]' else True)

<h2>Cleaning attributes</h2>
<p>
    After the changes we've made, we'll change the name of some attributes for consistency:
    <li>
        Because it now holds boolean values, <b>partner_name</b> will become <b>has_partner</b>
    </li>
    <li>
        <b>Title</b> and <b>tagline</b> now holds their length instead of a text, 
        and will be renames into <b>title_len</b> and <b>tagline_len</b>
    </li>
    <li>
        <b>cached_collected_pledges_count</b> name is just way too long and will be 
        simplified to <b>pledges_count</b>
    </li>
</p>

In [57]:
dataset = dataset.rename(columns={'cached_collected_pledges_count': 'pledges_count',
                        'partner_name': 'has_partner', 'title': 'title_len',
                        'tagline': 'tagline_len'})

<p>
    We also have many useless attributes which we'll drop.
    <ul>
        <li><b>igg_image_url, compressed_image_url, url, source_url and id</b>: are <b>unique identifiers</b> and thus are not suitabe for our needs</li>
        <li><b>amt_time_left and in_forever_funding</b>: are not relevant because we want to focus on <b>terminated projects</b> to see if they are funded or not. So in that case, amt_time_left will always be null and in_forever_funding will alwayse be false and it's irrelevant to base our prediction on data which as <b>always the same value</b>.</li>
        <li><b>balance</b>: is a duplication of the collected percentage in USD version</li>
        <li><b>category_name</b>: is a text version of category_slug. We don't need it twice</li>
        <li><b>nearest_five_percent</b>: seems to be the rounded collected_percentage value. As we have the collected percentage, this a uselss duplicate.</li>
    </ul>
</p>

In [58]:
cols_to_drop = ['nearest_five_percent', 'igg_image_url', 'compressed_image_url', 'url', 
                'category_url', 'category_name', 'amt_time_left', 'in_forever_funding',
                'source_url', 'id', 'balance']
dataset.drop(cols_to_drop, inplace=True, axis=1)

<p>Now our dataset look like this:</p>

In [59]:
dataset[:5]

,title_len,tagline_len,pledges_count,currency_code,category_slug,card_type,collected_percentage,has_partner,friend_contributors,friend_team_members,target_amount
2505,20,65,311,USD,video_web,project,181.0,False,False,False,4997.237569
3945,47,99,62,USD,technology,project,1178.0,False,False,False,2564.855688
46157,43,93,6704,USD,technology,project,2858.0,True,False,False,19997.445766
52031,15,19,1,USD,video_web,project,40.0,False,False,False,2500.000000
52599,6,97,29,USD,technology,project,39.0,False,False,False,30202.564103


<p>
    Now that we have removed the obvious unusable attributes, we'll check if the remaining attributes can be used properly.
</p>

In [60]:
dataset.currency_code.value_counts()

USD    116737
CAD      8028
GBP      6975
EUR      5977
AUD      1335
Name: currency_code, dtype: int64

<p>We can see that projects in <b>USD are a large majority</b> but it can be interesting to see if other currency are related to project succes.</p>

In [61]:
dataset.category_slug.value_counts()

film                    43239
music                   17116
local_businesses        15234
education               15052
dance_theater           10665
art                      7058
health_fitness           5951
writing_publishing       4175
animal_rights            2585
fashion_wearables        2447
environment              2411
local-businesses         1666
phones_accessories       1642
video_games              1374
comics                    999
writing-publishing        714
travel_outdoors           687
home                      684
dance-theater             664
photography               638
health-fitness            603
fashion-wearables         431
productivity              346
video-games               337
transportation            294
human_rights              274
animal-rights             251
phones-accessories        250
audio                     170
camera_gear               165
                        ...  
creative-works             30
tech_innovation            28
creative_w

<p>Here we can see 3 things:
    <ul>
        <li>Some slugs use '-' and other '_'. We'll have to normalize it</li>
        <li>There is <b>too much categories</b> and we must reduce this number to be able to use this attribute correctly</li>
        <li>Some categories contains <b>very few projects</b> (some categories have just one project). We'll probably use this during attribute reduction.</li>
    </ul>
    As category cleaning is a bit more complicated than the rest of the cleaning, it'll be processed on a dedicated notebook.
</p>

In [62]:
dataset.card_type.value_counts()

project    139052
Name: card_type, dtype: int64

In [63]:
dataset.friend_contributors.value_counts()

False    139052
Name: friend_contributors, dtype: int64

In [64]:
dataset.friend_team_members.value_counts()

False    139052
Name: friend_team_members, dtype: int64

<p><b>card_type, friend_team_members and friend_contributors</b> have only <b>one value</b>. This is unsuitable for our needs. Let's remove them.</p>

In [65]:
cols_to_drop = ['card_type', 'friend_team_members', 'friend_contributors']
dataset.drop(cols_to_drop, inplace=True, axis=1)

<p>Now our dataset look like this:</p>

In [66]:
dataset[:5]

,title_len,tagline_len,pledges_count,currency_code,category_slug,collected_percentage,has_partner,target_amount
2505,20,65,311,USD,video_web,181.0,False,4997.237569
3945,47,99,62,USD,technology,1178.0,False,2564.855688
46157,43,93,6704,USD,technology,2858.0,True,19997.445766
52031,15,19,1,USD,video_web,40.0,False,2500.000000
52599,6,97,29,USD,technology,39.0,False,30202.564103


<p>We have <b>8 attributes which is good</b>. Let's save it as a tsv file which we will use for all our analysis</p>

In [67]:
dataset.to_csv('indiegogo_cleaned_dataset.tsv', index=False, sep='\t')